In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
data = pd.read_csv('model_data.csv')
val_data = pd.read_csv('valid_data.csv')
val_data['url'] = val_data['url'].apply(str)

In [3]:
print(f'정상 url 비율 = {round(data["Label"].value_counts()[0]/len(data) * 100,3)}%')
print(f'피싱 url 비율 = {round(data["Label"].value_counts()[1]/len(data) * 100,3)}%')

정상 url 비율 = 51.725%
피싱 url 비율 = 48.275%


In [4]:
print(f'정상 url 비율 = {round(val_data["Label"].value_counts()[0]/len(val_data) * 100,3)}%')
print(f'피싱 url 비율 = {round(val_data["Label"].value_counts()[1]/len(val_data) * 100,3)}%')

정상 url 비율 = 71.429%
피싱 url 비율 = 28.571%


In [5]:
#독립 변수
X_data = data['url']

# 종속 변수
y_data = data['Label']

# 검증용 데이터
shuffled_val_data = val_data.sample(frac=1, random_state=0).reset_index(drop=True)
X_val = shuffled_val_data['url']
y_val = shuffled_val_data['Label']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

In [7]:
# 각 데이터의 크기 확인
print("훈련 데이터 크기:", X_train.shape, y_train.shape)
print("검증 데이터 크기:", X_val.shape, y_val.shape)
print("테스트 데이터 크기:", X_test.shape, y_test.shape)

훈련 데이터 크기: (231995,) (231995,)
검증 데이터 크기: (70000,) (70000,)
테스트 데이터 크기: (57999,) (57999,)


In [8]:
print('--------훈련 데이터의 비율-----------')
print(f'정상 url = {round(y_train.value_counts()[0]/len(y_train) * 100,3)}%')
print(f'스팸 url = {round(y_train.value_counts()[1]/len(y_train) * 100,3)}%')

print('--------테스트 데이터의 비율-----------')
print(f'정상 url = {round(y_test.value_counts()[0]/len(y_test) * 100,3)}%')
print(f'스팸 url = {round(y_test.value_counts()[1]/len(y_test) * 100,3)}%')

print('--------검증 데이터의 비율-----------')
print(f'정상 url = {round(y_val.value_counts()[0]/len(y_val) * 100,3)}%')
print(f'스팸 url = {round(y_val.value_counts()[1]/len(y_val) * 100,3)}%')

--------훈련 데이터의 비율-----------
정상 url = 51.725%
스팸 url = 48.275%
--------테스트 데이터의 비율-----------
정상 url = 51.725%
스팸 url = 48.275%
--------검증 데이터의 비율-----------
정상 url = 71.429%
스팸 url = 28.571%


In [9]:
# URL을 구성 요소로 분리하는 함수
def tokenize_url(url):
    parsed_url = urlparse(url)
    scheme = parsed_url.scheme
    netloc = parsed_url.netloc
    path = parsed_url.path

    # 구성 요소를 리스트로 결합
    url_parts = [scheme] + netloc.split('.') + path.split('/')

    # 빈 문자열 제거
    url_parts = [part for part in url_parts if part]

    return url_parts

# URL 리스트를 구성 요소로 분리
X_train_list = [tokenize_url(url) for url in X_train]

# 구성 요소를 문자열로 결합
X_train_list = [' '.join(parts) for parts in X_train_list]

# Tokenizer 초기화 및 텍스트 적합화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_list)

# 텍스트를 시퀀스로 변환
X_train_encoded = tokenizer.texts_to_sequences(X_train_list)

print("토큰화된 URL 구성 요소:", X_train_encoded[:5])

토큰화된 URL 구성 요소: [[1, 17, 10, 2, 22, 16, 14, 21, 28564, 28565, 28566, 8], [1, 28567, 39], [1, 27, 1261, 991, 1150, 5469, 5470, 27, 5], [1, 61, 2, 28568], [1, 59613, 38]]


In [10]:
word_to_index = tokenizer.word_index
word_to_index

{'https': 1,
 'com': 2,
 'ipfs': 3,
 'http': 4,
 'dev': 5,
 'kr': 6,
 'html': 7,
 'pub': 8,
 'www': 9,
 'google': 10,
 'cf': 11,
 'pages': 12,
 'r2': 13,
 'e': 14,
 'org': 15,
 'd': 16,
 'docs': 17,
 'net': 18,
 'go': 19,
 'weeblysite': 20,
 '2pacx': 21,
 'presentation': 22,
 'app': 23,
 'cloudflare': 24,
 'co': 25,
 'io': 26,
 'workers': 27,
 'web': 28,
 'php': 29,
 'es': 30,
 'webmail': 31,
 'network': 32,
 'firebaseapp': 33,
 'ru': 34,
 'eth': 35,
 'aragon': 36,
 'index': 37,
 'in': 38,
 'site': 39,
 'my': 40,
 'me': 41,
 'jp': 42,
 'top': 43,
 'cn': 44,
 'duckdns': 45,
 'login': 46,
 'new': 47,
 'link': 48,
 'ms': 49,
 'wixsite': 50,
 'adobe': 51,
 'express': 52,
 'weebly': 53,
 'webpage': 54,
 'id': 55,
 'or': 56,
 'att': 57,
 'gov': 58,
 'de': 59,
 'edu': 60,
 'tinyurl': 61,
 'hs': 62,
 'ir': 63,
 'blogspot': 64,
 'fr': 65,
 't': 66,
 'sites': 67,
 'info': 68,
 'view': 69,
 'home': 70,
 'wp': 71,
 'ac': 72,
 'page': 73,
 'pl': 74,
 'mail': 75,
 'online': 76,
 'br': 77,
 'myshopif

In [11]:
threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

등장 빈도가 1번 이하인 희귀 단어의 수: 110452
단어 집합(vocabulary)에서 희귀 단어의 비율: 64.9473139523944
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 9.86181212985392


In [12]:
vocab_size = len(word_to_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

단어 집합의 크기: 170065


In [13]:
print('url 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('url 평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))

url 최대 길이 : 560
url 평균 길이 : 4.827677


In [14]:
max_len = 560
X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
print("훈련 데이터의 크기(shape):", X_train_padded.shape)

훈련 데이터의 크기(shape): (231995, 560)


In [22]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 32
dropout_ratio = 0.3
num_filters = 32
kernel_size = 5

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.keras', monitor = 'val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[es, mc])

Epoch 1/10
2900/2900 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - acc: 0.9303 - loss: 0.2021
Epoch 1: val_acc improved from -inf to 0.98017, saving model to best_model.keras
2900/2900 ━━━━━━━━━━━━━━━━━━━━ 83s 28ms/step - acc: 0.9303 - loss: 0.2020 - val_acc: 0.9802 - val_loss: 0.0689
Epoch 2/10
2899/2900 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - acc: 0.9882 - loss: 0.0393
Epoch 2: val_acc improved from 0.98017 to 0.98871, saving model to best_model.keras
2900/2900 ━━━━━━━━━━━━━━━━━━━━ 80s 28ms/step - acc: 0.9882 - loss: 0.0393 - val_acc: 0.9887 - val_loss: 0.0420
Epoch 3/10
2899/2900 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - acc: 0.9977 - loss: 0.0085
Epoch 3: val_acc did not improve from 0.98871
2900/2900 ━━━━━━━━━━━━━━━━━━━━ 80s 28ms/step - acc: 0.9977 - loss: 0.0085 - val_acc: 0.9886 - val_loss: 0.0488
Epoch 4/10
2899/2900 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - acc: 0.9991 - loss: 0.0035
Epoch 4: val_acc improved from 0.98871 to 0.98983, saving model to best_model.keras
2900/2900 ━━━━━━━━━━━━━━━━━━━━ 82s 2

In [23]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))

1813/1813 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - acc: 0.9664 - loss: 0.1409

 테스트 정확도: 0.9671


In [24]:
X_val_encoded = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_encoded, maxlen = max_len)
print("\n 검증 데이터 정확도: %.4f" % (model.evaluate(X_val_padded, y_val)[1]))

2188/2188 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - acc: 0.9420 - loss: 0.2327

 검증 데이터 정확도: 0.9428
